In [1]:
import numpy as np
from collections import defaultdict
from scipy.sparse import csr_matrix, tril, triu, diags
import scipy.sparse.linalg as spla

def read_sparse_matrix_method1(filename):
    with open(filename, "r") as f:
        n = int(f.readline().strip())  # Read dimension
        d = np.zeros(n)  # Store diagonal elements
        sparse_rows = defaultdict(list)  # Store non-diagonal elements

        # For building CSR matrix
        data = []
        row_indices = []
        col_indices = []

        # Read all elements
        for line in f:
            parts = line.strip().split(',')
            if len(parts) == 3:
                value, row, col = map(float, parts)
                row, col = int(row), int(col)

                # Store in formats for both methods
                data.append(value)
                row_indices.append(row)
                col_indices.append(col)

                if row == col:
                    d[row] = value  # Diagonal element
                else:
                    sparse_rows[row].append((col, value))  # Off-diagonal element

        # Create CSR matrix
        A = csr_matrix((data, (row_indices, col_indices)), shape=(n, n))

    return n, d, sparse_rows, A

def check_diagonal_dominance(n, d, sparse_rows):
    """Check if the matrix is diagonally dominant"""
    is_diagonally_dominant = True
    worst_row_ratio = float('inf')
    problematic_rows = []

    for i in range(n):
        # Sum of absolute values of off-diagonal elements in row i
        row_sum = sum(abs(val) for col, val in sparse_rows[i])

        # Absolute value of diagonal element
        diag_abs = abs(d[i])

        # Check if diagonal element is zero
        if diag_abs == 0:
            print(f"Row {i}: Diagonal element is zero - Gauss-Seidel will fail")
            is_diagonally_dominant = False
            problematic_rows.append((i, 0, row_sum))
            worst_row_ratio = 0
            continue

        # Ratio of off-diagonal sum to diagonal element
        ratio = row_sum / diag_abs

        if ratio >= 1:
            is_diagonally_dominant = False
            problematic_rows.append((i, diag_abs, row_sum))

        # Track the worst row ratio
        if ratio < worst_row_ratio:
            worst_row_ratio = ratio

    return is_diagonally_dominant, worst_row_ratio, problematic_rows

def check_symmetric(A):
    """Check if the matrix is symmetric"""
    # Calculate A - A.T and check its norm
    diff = A - A.transpose()
    norm = spla.norm(diff, ord='fro')

    # If the norm is very small, consider the matrix symmetric
    epsilon = 1e-10
    is_symmetric = norm < epsilon

    return is_symmetric, norm

def estimate_spectral_radius(A):
    """Estimate the spectral radius of the iteration matrix for Gauss-Seidel"""
    # Extract diagonal, strictly lower and upper triangular parts
    D = diags(A.diagonal())
    L = tril(A, k=-1)
    U = triu(A, k=1)

    # Compute the iteration matrix: G = -(D+L)^(-1) * U
    try:
        DL_inv = spla.inv((D + L).tocsc())
        G = -DL_inv.dot(U)

        # Calculate some eigenvalues (not all for large matrices)
        max_eigs = min(100, A.shape[0]-1)
        eigenvalues = spla.eigs(G, k=max_eigs, return_eigenvectors=False)

        # Spectral radius is the maximum absolute eigenvalue
        spectral_radius = max(abs(eigenvalues))

        return spectral_radius, True
    except:
        # If matrix is singular or eigenvalue computation fails
        return None, False

def check_gauss_seidel_suitability(filename):
    """Check if a matrix is suitable for solving with Gauss-Seidel method"""
    print(f"Analyzing matrix in {filename} for Gauss-Seidel suitability...")

    # Read the matrix
    n, d, sparse_rows, A = read_sparse_matrix_method1(filename)
    print(f"Matrix size: {n}x{n}")

    # Check diagonal dominance
    is_diag_dominant, worst_ratio, problematic_rows = check_diagonal_dominance(n, d, sparse_rows)

    if is_diag_dominant:
        print("✓ Matrix IS diagonally dominant - Gauss-Seidel will converge")
        print(f"  Worst row ratio (off-diagonal sum / diagonal): {worst_ratio:.6f}")
    else:
        print("✗ Matrix is NOT diagonally dominant - Gauss-Seidel may diverge")
        print(f"  Worst row ratio (off-diagonal sum / diagonal): {worst_ratio:.6f}")
        print(f"  Number of problematic rows: {len(problematic_rows)}")

        # Print some examples of problematic rows
        if len(problematic_rows) > 0:
            print("  Examples of problematic rows (row_index, |diagonal|, off_diagonal_sum):")
            for i, (row, diag, off_sum) in enumerate(problematic_rows[:5]):
                print(f"    Row {row}: |diagonal| = {diag:.2e}, off-diagonal sum = {off_sum:.2e}")
            if len(problematic_rows) > 5:
                print(f"    ... and {len(problematic_rows) - 5} more problematic rows")

    # Check symmetry
    is_symmetric, sym_diff = check_symmetric(A)
    if is_symmetric:
        print("✓ Matrix IS symmetric")
        # For symmetric matrices, check positive definiteness
        # Note: This is computationally expensive for large matrices
        if n <= 5000:  # Only do this check for reasonably sized matrices
            try:
                min_eig = spla.eigsh(A, k=1, which='SA', return_eigenvectors=False)[0]
                if min_eig > 0:
                    print("✓ Matrix IS positive definite - Gauss-Seidel will converge")
                else:
                    print("✗ Matrix is NOT positive definite - Gauss-Seidel may diverge")
                print(f"  Smallest eigenvalue: {min_eig:.6e}")
            except:
                print("? Could not determine if matrix is positive definite")
    else:
        print(f"✗ Matrix is NOT symmetric (difference norm: {sym_diff:.6e})")

    # Estimate spectral radius of iteration matrix
    if n <= 5000:  # Only for reasonably sized matrices
        spectral_radius, success = estimate_spectral_radius(A)
        if success:
            if spectral_radius < 1:
                print(f"✓ Spectral radius = {spectral_radius:.6f} < 1 - Gauss-Seidel will converge")
                # Estimate number of iterations for convergence
                error_reduction = 1e-8  # Reduce error by this factor
                iterations = int(np.ceil(np.log(error_reduction) / np.log(spectral_radius)))
                print(f"  Estimated iterations for convergence: ~{iterations}")
            else:
                print(f"✗ Spectral radius = {spectral_radius:.6f} >= 1 - Gauss-Seidel will diverge")
        else:
            print("? Could not estimate spectral radius")

    # Final verdict
    print("\nVERDICT:")
    if is_diag_dominant or (is_symmetric and min_eig > 0 if 'min_eig' in locals() else False):
        print("This matrix IS suitable for Gauss-Seidel method.")
        return True
    else:
        print("This matrix is likely NOT suitable for Gauss-Seidel method.")
        print("Recommended alternatives: Direct solver, GMRES, or BiCGSTAB")
        return False

if __name__ == "__main__":
    # Run the check on your matrix
    filename = "a_5.txt"
    is_suitable = check_gauss_seidel_suitability(filename)

Analyzing matrix in a_5.txt for Gauss-Seidel suitability...
Matrix size: 2025x2025
✗ Matrix is NOT diagonally dominant - Gauss-Seidel may diverge
  Worst row ratio (off-diagonal sum / diagonal): 0.025641
  Number of problematic rows: 1935
  Examples of problematic rows (row_index, |diagonal|, off_diagonal_sum):
    Row 0: |diagonal| = 2.62e+01, off-diagonal sum = 3.62e+01
    Row 2: |diagonal| = 7.25e+00, off-diagonal sum = 1.72e+01
    Row 3: |diagonal| = 5.02e+01, off-diagonal sum = 6.02e+01
    Row 4: |diagonal| = 2.70e+01, off-diagonal sum = 3.70e+01
    Row 5: |diagonal| = 5.00e-01, off-diagonal sum = 9.50e+00
    ... and 1930 more problematic rows
✗ Matrix is NOT symmetric (difference norm: 1.149629e+03)
✗ Spectral radius = 4.581446 >= 1 - Gauss-Seidel will diverge

VERDICT:
This matrix is likely NOT suitable for Gauss-Seidel method.
Recommended alternatives: Direct solver, GMRES, or BiCGSTAB
